# Plotter of pearl trajectory

## Goals of this notebook

Main goal - create a calculator that uses cannon construction and target displacement to calculate number of wind charges

## Sources

Great page with math: https://minecraft.wiki/w/Tutorial:360_degree_ender_pearl_cannon


## Static data

In [ ]:
import numpy as np
from enum import Enum

# Will use Minecraft notation - X is 'right', Y is 'up', Z is 'toward me'

""" 
Trajectory planning is solved. Need to process directions and offsets.

User inputs: item frame facing direction, item frame position, target position. All absolute, world reference frame.

Stages of algorythm:
- Get global position of pearl launch area
    Take default offset, shift to new direction, add to item frame position
- Get relative position of target in global frame
- Rotate relative position into cannon frame
- Get UI quadrant and correct charge pair

"""

# Enum for Minecraft world directions. East is +X, South is +Z.
class cardinalDir(Enum):
    NORTH = 0
    EAST = 1
    SOUTH = 2
    WEST = 3

# Array of wind charge positions relative to launch zone center. 
# Assumes that charge stacks enter in bottom-left corner (from East / positive X). 
# Also assumes that explosion center is at zero relative height (for now)
# Coordinates: +X is left, +Y is vertical up, +Z is forward (away from camera)
chargePositions = np.array([
    [+0.83375, 0.75, +0.16625],      # earliest pos in spiral
    [-0.16625, 0.75, +0.84375], 
    [-0.84375, 0.75, -0.16625], 
    [+0.16625, 0.75, -0.84375]       # latest pos in spiral (gets inside 1st block)
    ])

# Facing direction of cannon in default orientation. 
# Taken by looking at item frame
defaultFacingDir = cardinalDir.WEST

# Offset from block behind item frame to center of 2x2 explosion area
# In default facing direction, expPos = itemFramePos + offset
itemToOriginOffset = np.array([-28, 14, -17])

# Coordinate of ender pearl (feet pos) at the moment of explosion, relative to 2x2 area center. 
# Same orientation as for wind charge coordinates
# (for now assume it's in top-right quadrant)
pearlPosition = np.array([-0.0625, 0.540222006, +0.0625])




## Technical functions
Functions for movement and orientation calculations

In [ ]:
import numpy as np
from enum import Enum
from scipy.optimize import fsolve


def rotateCoord(startDir: cardinalDir, endDir: cardinalDir, coordVec: np.ndarray) -> np.ndarray:
    """
    Rotate coordinate by azimuth from starting to final cardinal direction. 
    Example:
        rotateCoord(NORTH, EAST, [1, 0, 0]) => [0, 0, 1]
    TODO - test use on array of vectors (Nx3 matrix)

    Args:
        startDir (cardinalDir): initial facing direction
        endDir (cardinalDir): final facing direction
        coordVec (np.ndarray): 1x3 vector to be rotated

    Returns:
        np.ndarray: 1x3 vector facing new direction
    """
    angleCCW = (endDir.value - startDir.value) * 90
    angleRad = np.deg2rad(angleCCW)
    rotMatrix = np.eye(3)
    rotMatrix[0][0] = rotMatrix[2][2] = np.cos(angleRad)
    rotMatrix[0][2] = np.sin(angleRad)
    rotMatrix[2][0] = -np.sin(angleRad)
    return np.matmul(coordVec, rotMatrix)



# # Only valid for ender pearls due to ticking order
# def projectileTickStep(pos, vel):
#     # acceleration
#     vel += np.array([0.0, -0.03, 0.0])
#     # drag
#     vel *= 0.99
#     # position update
#     pos += vel
#     return (pos, vel)

def pearlInitVelFromEndPos(tick: int, displacement: np.ndarray) -> np.ndarray:
    """
    Get initial velocity to get required ender pearl displacement after some time
    (see wiki for formula)

    Args:
        tick (int): flight time from trajectory start in gameticks
        displacement (np.ndarray): target displacement as 1x3 vector

    Returns:
        np.ndarray: initial velocity as 1x3 vector
    """
    
    d = 0.9900000095367432
    a = np.array([0.0, -0.03, 0.0])
    return (1 - d) / (d * (1 - pow(d, tick))) * (displacement - d * tick * a / (1 - d)) + (d / (1 - d) * a)

def pearlPosFromInitVel(tick: float, vel0: np.ndarray) -> np.ndarray:
    """
    Get ender pearl displacement from initial velocity after some time
    (see wiki for formula)

    Args:
        tick (float): time from trajectory start in gameticks
        vel0 (np.ndarray): initial velocity as 1x3 vector

    Returns:
        np.ndarray: displacement relative to starting point as 1x3 vector
    """

    d = 0.9900000095367432
    a = np.array([0.0, -0.03, 0.0])
    return (d * (1 - pow(d, tick))) / (1 - d) * (vel0 - d / (1 - d) * a) + (d * tick * a / (1 - d))

def getExplosionMatrix(velA:np.ndarray, velB:np.ndarray) -> np.ndarray:
    """
    Get E matrix from two 3D explosion velocity vectors
    (see tutorial wiki page)

    Args:
        velA (np.ndarray): velocity from first explosion as 1x3 vector
        velB (np.ndarray): velocity from second explosion as 1x3 vector

    Returns:
        np.ndarray: 2x3 E matrix
    """
    
    velMatrix = np.transpose(np.vstack((velA, velB)))
    A = np.matmul(np.transpose(velMatrix), velMatrix)
    B = np.linalg.inv(A)
    C = np.matmul(B, np.transpose(velMatrix))
    return C



# def getPearlVelAndTime(targetPos: np.ndarray) -> tuple[int, np.ndarray]:
#     """
#     Get velocity vector and flight time to hit relative target position. 
#     Assumes negligible initial vertical velocity
#     (not very good assumption, will work on more complex optimizer)

#     Args:
#         targetPos (np.ndarray): target position as displacement relative to initial position

#     Returns:
#         tuple[int, np.ndarray]: flight time in ticks and 
#     """
#     time = findFallTime(targetPos[1])
#     if time == 0:
#         return (0, np.zeros(3))
#     initVel = pearlInitVelFromEndPos(time, targetPos)
#     return (time, initVel)

# TODO - standardise directions
def getRelOffset(pearlPos: np.ndarray, targetPos: np.ndarray, facingAzimuth: int) -> np.ndarray:
    """
    Get target offset in cannon's frame of reference

    Args:
        pearlPos (np.ndarray): absolute position of ender pearl at the moment of acceleration
        targetPos (np.ndarray): absolute position of target
        facingAzimuth (int): direction of cannon - azimuth when looking from pearl to charge generator. Units - degrees, multiple of 90.

    Returns:
        np.ndarray: relative target offset in cannon's frame of reference
    """
    relDisp = targetPos - pearlPos
    
    # Rotate displacement according to cannon facing direction.
    # Cannon frame of reference will point to East (line from pearl in direction where charge stacks enter from)
    rotAngleRad = np.radians(-facingAzimuth)
    rotMatrix = np.eye(3)
    rotMatrix[0][0] = rotMatrix[2][2] = np.cos(rotAngleRad)
    rotMatrix[0][2] = np.sin(rotAngleRad)
    rotMatrix[2][0] = -np.sin(rotAngleRad)
    dispRotated = np.matmul(relDisp, rotMatrix)
    return dispRotated

def getChargePearlPushVelocity(explosionCenterPos: np.ndarray, pearlFeetPos: np.ndarray) -> np.ndarray:
    """
    Calculate impulse to ender pearl from one wind charge explosion. 
    Assumes 100% exposure 
    (see Explosion wiki page for formulas)

    Args:
        explosionCenterPos (np.ndarray): position of wind charge explosion as 1x3 vector
        pearlFeetPos (np.ndarray): position of ender pearl feet (reported position) as 1x3 vector

    Returns:
        np.ndarray: velocity change as 1x3 vector (away from wind charge)
    """
    
    feetDist = pearlFeetPos - explosionCenterPos
    PearlEyeHeight = 0.2125
    WindChargePower = 3.0
    exposure = 1.0    
    magnitude = (1 - np.linalg.norm(feetDist) / (2 * WindChargePower)) * exposure
    eyeVector = feetDist + np.array([0.0, PearlEyeHeight, 0.0])
    velocityVector = eyeVector / np.linalg.norm(eyeVector) * magnitude
    return velocityVector

def findChargeAmount(velA: np.ndarray, velB: np.ndarray, targetOffs: np.ndarray) -> np.ndarray:
    """
    Solve for wind charge stack sizes and flight time

    Args:
        velA (np.ndarray): velocity added to ender pearl from one wind charge in position A. Format - 1x3 vector
        velB (np.ndarray): velocity added to ender pearl from one wind charge in position B. Format - 1x3 vector
        targetOffs (np.ndarray): relative offset from initial pearl position to target

    Returns:
        np.ndarray: nearest solution as 1x3 vector of floats. 
            Element 0 - flight time (in gameticks). Element 1 - size of charge stack A. Element 2 - size of charge stack B.
    """
    
    # idea - use gameticks as another variable, 3 total. Return full pos error
    def optimizeFunc(x):
        tick, amtA, amtB = x
        velVector = amtA * velA + amtB * velB
        pearlPos = pearlPosFromInitVel(tick, velVector)
        posError = targetOffs - pearlPos
        return posError
    result = fsolve(optimizeFunc, [1, 0, 0])
    
    return result # type: ignore

def getLocalQuadData(targetOffs : np.ndarray) -> tuple[launchQuadId, np.ndarray, np.ndarray]:
    """
    Get shooting quadrant info from target position offset

    Args:
        targetOffs (np.ndarray): target position relative to acceleration zone in cannon-local frame

    Returns:
        tuple[launchQuadId, np.ndarray, np.ndarray]: id of quad for UI, position of first charge stack, position of last charge stack
    """
    
    # Tested, adjusted, seems to give correct results.
    
    chargePearlVectors = chargePositions - pearlPosition
    
    # Add target displacement into same array for batch processing
    allVectorArr = np.vstack((chargePearlVectors, -targetOffs))
    chargeAngles = np.arctan2(-allVectorArr[:, 2], allVectorArr[:, 0])
    simpleAnglesDeg = np.mod(np.rad2deg(chargeAngles) + 360, 360)
    targetAngleDeg = simpleAnglesDeg[4]
    
    # Find nearest angle in CCW direction
    ccwNearestIndex = 0
    # Get all angles bigger (more CCW) than target angle
    cwAngles = [(i, ang) for i, ang in enumerate(simpleAnglesDeg[0:4]) if ang > targetAngleDeg]
    # Get nearest CCW angle to target
    if len(cwAngles) == 0:
        ccwNearestIndex = 0
    else:
        ccwNearestIndex = min(cwAngles, key=lambda x: x[1])[0]
    
    cwNearestIndex = (ccwNearestIndex + 1) % 4
    quadId = launchQuadId(cwNearestIndex)
    firstStackCoord = chargePositions[max(cwNearestIndex, ccwNearestIndex)]
    secondStackcoord = chargePositions[min(cwNearestIndex, ccwNearestIndex)]
    
    return (quadId, firstStackCoord, secondStackcoord)


## Testing area

In [ ]:
import numpy as np
from enum import Enum



# Support functions for orientation adjustments

# More readable indexing for launch area quadrants. 
# Directions are in cannon-local frame
class launchQuadId(Enum):
    NE = 0
    SE = 1
    SW = 2
    NW = 3

# Test of orientation adjustments

frameFacingDir = cardinalDir.WEST
frameBlockPos = np.array([-81, 84, -90])
targetPos = np.array([-250, 55, -500])

originPos = frameBlockPos + rotateCoord(defaultFacingDir, frameFacingDir, itemToOriginOffset)
globalTargetOffset = targetPos - originPos
localTargetOffset = rotateCoord(frameFacingDir, defaultFacingDir, globalTargetOffset)







# Test of trajectory solver

targetOffs = localTargetOffset
quadId, stackPosA, stackPosB = getLocalQuadData(targetOffs)
velA = getChargePearlPushVelocity(stackPosA, pearlPosition)
velB = getChargePearlPushVelocity(stackPosB, pearlPosition)
idealCannonSetting = findChargeAmount(velA, velB, targetOffs)
# Get realistic values, check rounding error
ticks, chargesA, chargesB = idealCannonSetting.round()
velVector = chargesA * velA + chargesB * velB
pearlPos = pearlPosFromInitVel(ticks, velVector)
print(f"Target offset: {targetOffs}; Real offset: {pearlPos}")
print(f"Quad ID: {quadId.value}; Flight time: {ticks}; Charges A: {chargesA}; Charges B: {chargesB}")




Target offset: [-141.  -43. -393.]; Real offset: [-137.29725257  -43.14067503 -405.80098166]
Quad ID: 1; Flight time: 60.0; Charges A: 10.0; Charges B: 5.0
